In [1]:
# -*- coding: utf-8 -*-

import numpy as np
import matplotlib.pyplot as plt
import wave

In [2]:
def readWave(filepath):
    wf = wave.open(filepath, "r")
    fs = wf.getframerate()
    x = wf.readframes(wf.getnframes())
    x = np.frombuffer(x, dtype="int16") / 32768.0  # (-1, 1)に正規化
    wf.close()
    return x, float(fs)

In [3]:
a,b = readWave("../data/LearningData/a.wav")
print a,b
print a[0]

[ 0.00073242  0.00082397  0.00088501 ... -0.00024414 -0.00015259
 -0.00018311] 44100.0
0.000732421875
